# 抽象数据类型(Abstract Data Type, ADT)
基本思想：把数据定义为抽象的对象集合，只为它们定义可用的合法操作，不暴露内部实现的具体细节。  
数据类型的操作：  
(1)构造操作：基于已知信息，产生同类型的新对象，如基于两个有理数产生表示它们之和的有理数对象；  
(2)解析操作：从对象中取得有用的信息，如从一个有理数获取分子或分母；  
(3)变动操作：修改被操作对象的内部状态  
Python语言中，str、tuple和frozenset是不变数据类型，仅可实现构造和解析操作，list、set和dict是可变数据类型  
## -----------------------------------------------------------------------------------------------------------------------------------

# Python的类

## 有理数类

In [3]:
# 例：有理数类
class Rational0:
    def __init__(self, num, den = 1):
        self.num = num
        self.den = den
    
    def plus(self, another):
        den = self.den * another.den
        num = self.num * another.den + self.den * another.num
        return Rational0(num, den)
    
    def print(self):
        print(str(self.num) + "/" + str(self.den))       

代码说明：  
操作类最常用的方式是实例方法，这种方法总是从本类的对象出发去调用，参数表中第一个参数表示实际使用的调用对象，通常以self作为参数名。上面Rational0中定义了三个实例方法。  
在一个类中，通常会定义一个名为\__init\__的方法，其工作是构造本类的新对象。创建实例对象采用函数调用的描述形式，以类名作为函数名，系统将建立该类的新对象，并自动对该对象执行\__init\__方法，如r1 = Rational0(3, 5),就是要求创建一个值为3/5的有理数对象，并将此对象赋给变量r1，调用式应给出除self之外的其他实际参数。

In [4]:
# 调用
r1 = Rational0(3, 5)
r2 = r1.plus(Rational0(7, 15))
r2.print()

80/75


上述程序输出结果：80/75，显然尽管正确但不是合适的形式，若有理数更为复杂，得到的结果会变得越来越大，因此还需进一步考虑

## 类定义进阶
在类的定义中，有下划线\_开头的属性名（和函数名）都当做内部使用的名字，不应在类外使用。另外，Python对类定义里以两个下划线开头(但不以两个下划线结尾)的名字做了特殊处理，使得在类定义之外不能直接用这个名字访问。  
考虑上述有理数化简问题，在建立有理数是，需约去最大公约数，因而需定义求最大公约数的函数gcd。gcd参数应为两个整数，并且不依赖与任何有理数类的对象，因此其参数表中似乎不应该以表示有理数的self作为第一个参数。但是，gcd是为了有理数的实现而需要使用的一种辅助功能，局部使用的功能不应定义为全局函数。因此，gcd应是在有理数类中定义的一个非实例函数。  
Python把在类中定义的这种方法称为静态方法(与实例方法不同)，描述说需要在函数定义的头部加修饰符@staticmethod。静态方法中的参数表不应有self参数。  
另外，其他需要考虑的问题：分母不为0；对于正负，保证分母为正，用分子的正负表示有理数的正负。

In [ ]:
# 修改后的程序,部分
class Rational:
    @staticmethod
    def _gcd(m,n):
        if n == 0:
            m, n = n, m
        while m != 0:
            m, n = n % m, m
        return n
    
    def __init__(self, num, den = 1):
        if not isinstance(num, int) or not isinstance(den, int):
            raise TypeError
        if den == 0:
            raise ZeroEivisionError
        sign = 1
        if num < 0:
            num, sign = -num, -sign
        if den < 0:
            den, sign = -den, -sign
        g = Rational._gcd(num, den)
        self._num = sign * (num//g)
        self._den = den//g
'''
# 其他函数可补充，如
def num(self): return self._num
def den(self): return self._den
其他：上述加法用了plus名字，可改用__add__，其他减、乘、除，比较大小等也可以用此方式
'''

其他函数可补充，如  
def num(self): return self._num  
def den(self): return self._den  
上述加法用了plus名字，可改用\__add\__，其他减、乘、除，比较大小等也可以用此方式  
检查类型用isinstance(变量，类型)，检查是否为自定义的有理数，可用isinstance(变量，Rational)  
## -----------------------------------------------------------------------------------------------------------------------------------

# 类的定义与使用
## 静态方法与类方法
静态方法，如前所示加修饰符@staticmethod，例如在Rational中的\_gcd，可用self.\_gcd(...)或Rational.\_gcd(...)调用，但需注意，静态方法无self参数，需为参数表中每个形参提供实参  
类方法：加修饰符@classmethod，此方法必须有一个表示其调用类的参数，习惯用cls作为参数名，还可以有其他参数。类方法执行时，调用它的类将自动约束到方法的cls参数，可以通过此参数访问该类的其他属性。

In [ ]:
# 例：类方法
class Countable:
    counter = 0
    
    def __init__(self):
        Countable.counter += 1
    
    @classmethod
    def get_count(cls):
        return Countable.counter
    
x = Countable()
y = Countable()
z = Countable()

print(Countable.get_count())

上述程序输出结果为3。为了记录本类创建的对象个数，Countable类里定义了一个数据属性counter，其值设置为0.每次创建这个类的对象时，初始化方法\__init\__就会吧这个对象计数器加1.类方法get_count访问了这个数据属性，上面程序片段在运行时将输出整数3，表示执行到print语句为止已经创建了3个Countable对象。  

## 私有变量
python内部无定义私有变量的机制，习惯约定时把以一个下划线开头的名字作为实例对象内部的对象，不从对象外部访问。以两个下划线开头，不以两个下划线结尾的属性，外部访问无法找到；以两个下划线开头和结尾的属性具有特殊含义。  

## 继承
继承作用：一是重复利用已有代码，减少定义新类的工作量，简化新功能的开发；另一作用是建议一组类之间的继承关系，利用这种关系更好组织和构造复杂的程序。  
通过继承定义出的新类被称为**派生类**(或**子类**)，被继承的已有类被称为此派生类的**基类**(或**父类**)。Python用内置函数issubclass检查两个类是否有继承关系。

In [ ]:
# 例：继承字符串类
class MyStr(str):
    pass

s = MyStr(1234)
issubclass(MyStr, str)#True
issubclass(s, MyStr)#True
issubclass(s, str)#True

派生类常需要重新定义\__init\__函数，完成该类实例的初始化。派生类\__init\__定义的常见形式：  

    class DerivedClass(BaseClass):  
        def __init__(self, ...):  
            BaseClass.__init__(self, ...)  
            ...... # 初始化函数的其他操作  
        ...... # 派生类的其他操作  

这里继承BaseClass类定义的派生的DerivedClass类。在调用基类的初始化方法时，必须明确写出基类的名字，不能从self出发去调用。在调用基类的\__init\__时，必须把表示本类对象的self作为调用的第一个实参，可能还需要传入其他的实参。在派生类里覆盖基类定义的函数时，可用BasClass.methodName(...)调用基类方法。当然不管是否被覆盖，都可以用此种方式调用，但需要此种方式通常需要把表示本对象的self参数作为第一个实参。  

## 方法查找
方法查找时，先查找当前类的属性，若无则再到调用的基类中寻找，全部无时报AttributeError异常。  

In [ ]:
# 例
class B:
    def f(self):
        self.g()
    def g(self):
        print('B.g called')

class C(B):
    def g(self):
        print('C.g called')

如果创建B类的实例对象x后调用x.f(),输出“B.g called”  
如果创建C类的实例对象y后调用y.f(),由于C中无f的定义，实际调用的时B中的f，**此时调用哪一个g会根据当时self所表示的实例对象的类型决定(动态绑定/约束)，输出“C.g called”**，根据静态程序正文去确定被调用方法的规则称为静态约束/静态绑定。  

## 标准函数super()
super函数定义在派生类中，要求从基类开始做属性检索。可写出基类名字或者不写出，写出基类时总是调用基类的方法，如果只写super(),会根据当时类的情况去找到相应的基类，自动确定使用哪个基类的属性。  
super有多种使用方式，最简单的时不带参数的调用，如super().m1(...),检索函数m1。